##### Parse the artist conent and Observe the aritists poularity, see if it make sense

In [1]:
import json
from bson.json_util import dumps
import pandas as pd
from fastparquet import ParquetFile,write
import time

In [2]:
pf=ParquetFile('artists.parq')
art=pf.to_pandas()

In [4]:
art.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41865 entries, 0 to 41864
Data columns (total 2 columns):
event_id       41865 non-null int64
artist_json    41865 non-null object
dtypes: int64(1), object(1)
memory usage: 654.2+ KB


In [3]:
art.iloc[11,0]

11163738

In [6]:
e_art_json=art['artist_json']

In [7]:
e_art_json[15891]  #check an example of artist info

[{'id': 2067881,
  'title': 'Harvey McLaughlin',
  'permalink': '/artists/harvey-mclaughlin',
  'data_path': '/artists/2067881/quickie',
  'description': '\'Tabloid News\' is the debut solo album by San Antonio born and based singer and songwriter Harvey McLaughlin. The album represents something of a throwback to the 1970\'s piano based long players by the likes of greats such as Warren Zevon, Randy Newman and Tom Waits with nods to New Orleans R&amp;B and Chicano Soul while maintaining the urgency of the young roots rock bands bands that Harvey has honed his craft and musicianship with over the last dozen years. He only half-jokingly admits "I want to play the piano the way Link Wray plays the guitar."\r\n\r\nThematically the album draws on McLaughlin\'s fascination with pulp magazines and urban legends as seen through the eyes of a young but weary romantic. McLaughlin says “I’ve made a career out of writing about dragons, ghost trains, dodo birds, banshee queens, chupacabras, and ot

In [8]:
#create a artist library
artist_rec=pd.DataFrame(columns=['name','do512_id','popularity','nev_on512','on_youtube','on_spotify','hometown'])

art_id=[]
art_name=[]  # get the event counts for per artist
art_pop=[]
art_yt=[]
art_sptf=[]
art_ht=[]

for item in e_art_json:
    if len(item)!=0:

        if len(item)==1:
            dart=json.loads(dumps(item))[0]
            art_id+=[dart['id']]
            art_name+=[dart['title']]
            art_pop+=[dart['popularity']]
            art_ht+=[dart['hometown']]
            if 'youtube_id' in dart.keys():
                if (dart['youtube_id']) and (len(dart['youtube_id'])>0):
                    art_yt+=[True]
                else:
                    art_yt+=[False]
            if 'spotify_id' in dart.keys():
                if (dart['spotify_id']) and (len(dart['spotify_id'])>0):
                    art_sptf+=[True]
                else:
                    art_sptf+=[False]
        else:
            for sub in item:
                if len(sub)!=0:
                    dart=json.loads(dumps(sub))
                    art_id+=[dart['id']]
                    art_name+=[dart['title']]
                    art_pop+=[dart['popularity']]
                    art_ht+=[dart['hometown']]
                    if 'youtube_id' in dart.keys():
                        if (dart['youtube_id']) and (len(dart['youtube_id'])>0):
                            art_yt+=[True]
                        else:
                            art_yt+=[False]
                    if 'spotify_id' in dart.keys():
                        if (dart['spotify_id']) and (len(dart['spotify_id'])>0):
                            art_sptf+=[True]
                        else:
                            art_sptf+=[False]

In [9]:
artist_rec['name']=art_name
artist_rec['do512_id']=art_id
artist_rec['popularity']=art_pop
artist_rec['hometown']=art_ht
artist_rec['on_youtube']=art_yt
artist_rec['on_youtube']=art_sptf

In [10]:
s=artist_rec.name.value_counts()

In [14]:
# put the event counts back to the dataframe
for artist in s.index.values:
    artist_rec.loc[artist_rec['name']==artist,'nev_on512']=s[artist]

In [15]:
artist_rec.drop_duplicates(subset='do512_id',keep=False,inplace=True)

In [16]:
# sort artists by popularity: show the popularity by the popularity rating on 512
artist_rec.sort_values(by='popularity',axis=0, ascending=False).head(100)

,name,do512_id,popularity,nev_on512,on_youtube,on_spotify,hometown
7849,Young The Giant,99129,108669.0,1,True,NaN,
1670,Ben Folds,4000,100142.0,1,True,NaN,
7850,Fitz and The Tantrums,214064,97652.0,1,True,NaN,Los Angeles
33128,Brian Wilson,105518,94709.0,1,True,NaN,"Los Angeles, CA"
30354,The Flaming Lips,34248,79420.0,1,True,NaN,"Oklahoma City, Oklahoma"
15872,Ludacris,71998,75858.0,1,True,NaN,
27203,Metric,54101,73501.0,1,True,NaN,"Toronto, NYC, Oakland"
9470,LOCAL NATIVES,49854,65236.0,1,True,NaN,"Los Angeles, CA"
13332,Yeah Yeah Yeahs,32566,52488.0,1,True,NaN,"New York, NY"
36208,TV On The Radio,3416,51489.0,1,True,NaN,


In [47]:
evvt=pd.read_csv('event_votes.csv')
evvt=evvt.set_index('id')
evvt['evid']=evvt.index.values
evvt.drop_duplicates(subset='evid',inplace=True)

In [57]:
# add votes column to artist_record
artist_rec['ev_votes']=[[] for x in range(len(artist_rec))]

In [58]:
# link artist popularity by the events they went and the event vote
# loop through the artist json for the index on event id
for i,item in enumerate(e_art_json):
    art_ind_evvot=[]
    if len(item)!=0:
        if len(item)==1:
            art_id=json.loads(dumps(item))[0]['id'] # get artist id
            # get event id
            evid=art.iloc[i,0]
            # get event votes
            evvotes=evvt.loc[evid,:][0]
            artist_rec.loc[artist_rec['do512_id']==art_id,'ev_votes']=artist_rec.loc[artist_rec['do512_id']==art_id,'ev_votes']+[evvotes] # save votes back to artist
        else:
            for sub in item:
                if len(sub)!=0:
                    art_id=json.loads(dumps(sub))['id'] # get artist id
                     # get event id
                    evid=art.iloc[i,0]
                    # get event votes
                    evvotes=evvt.loc[evid,:][0]
                    artist_rec.loc[artist_rec['do512_id']==art_id,'ev_votes']=artist_rec.loc[artist_rec['do512_id']==art_id,'ev_votes']+[evvotes] # save votes back to artist 
                                                       
            
            

In [50]:
artist_rec.loc[artist_rec['do512_id']==48401,'ev_votes']=artist_rec.loc[artist_rec['do512_id']==48401,'ev_votes']+[1]

In [55]:
artist_rec.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8212 entries, 12 to 40386
Data columns (total 8 columns):
name          8212 non-null object
do512_id      8212 non-null int64
popularity    8212 non-null float64
nev_on512     8212 non-null int64
on_youtube    8212 non-null bool
on_spotify    0 non-null object
hometown      4368 non-null object
ev_votes      8212 non-null object
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 521.3+ KB


In [61]:
artist_rec=artist_rec.sort_values(by='popularity',axis=0, ascending=False)

In [62]:
artist_rec.head(50)

,name,do512_id,popularity,nev_on512,on_youtube,on_spotify,hometown,ev_votes
7849,Young The Giant,99129,108669.0,1,True,NaN,,[739]
1670,Ben Folds,4000,100142.0,1,True,NaN,,[479]
7850,Fitz and The Tantrums,214064,97652.0,1,True,NaN,Los Angeles,[739]
33128,Brian Wilson,105518,94709.0,1,True,NaN,"Los Angeles, CA",[29]
30354,The Flaming Lips,34248,79420.0,1,True,NaN,"Oklahoma City, Oklahoma",[623]
15872,Ludacris,71998,75858.0,1,True,NaN,,[55]
27203,Metric,54101,73501.0,1,True,NaN,"Toronto, NYC, Oakland",[316]
9470,LOCAL NATIVES,49854,65236.0,1,True,NaN,"Los Angeles, CA",[821]
13332,Yeah Yeah Yeahs,32566,52488.0,1,True,NaN,"New York, NY",[140]
36208,TV On The Radio,3416,51489.0,1,True,NaN,,[354]


In [63]:
artist_rec.to_csv('Do512Artists.csv')